In [6]:
import geojson
import json
import time, math

import rasterio
import geopandas as gpd
import pandas as pd
import numpy as np
import GOSTrocks.rasterMisc as rMisc

from GOSTrocks.misc import tPrint

from scipy import stats
from scipy import ndimage
from scipy.ndimage import generic_filter
from scipy.sparse.csgraph import connected_components
from rasterio import features
from rasterio.features import rasterize
from shapely.geometry import shape, Polygon
from geopy.geocoders import Nominatim

In [2]:
input_pop_file = 'C:/WBG/Work/Projects/NGA_Urbanization/Data/Population\\nga_pop_2019_CN_100m_R2025A_v1.tif'
inR = rasterio.open(input_pop_file)

In [ ]:
densVal=3
totalPopThresh=5000
smooth=False
verbose=False
queen=False
raster=""
raster_pop=""
print_message=""

popRaster = inR
data = popRaster.read()
urbanData = (data > densVal) * 1
urbanData = urbanData.astype("int16")



13.051555476320038

In [ ]:
sieve_size = round(totalPopThresh/data.max()) # the minimum number of pixels to reach the totalPopThresh at max density
sieve_size * 2
urban_raster_sieved = features.sieve(urbanData, size=sieve_size)
idx = 0
for cShape, value in features.shapes(urban_raster_sieved, transform=popRaster.transform):
    idx +=1

print(idx)

120606


In [12]:
sieve_size = round(totalPopThresh/data.max()) # the minimum number of pixels to reach the totalPopThresh at max density
sieve_size = sieve_size * 2
urban_raster_sieved = features.sieve(urbanData, size=sieve_size)
idx = 0
for cShape, value in features.shapes(urban_raster_sieved, transform=popRaster.transform):
    idx +=1
    if value == 1:
         break

print(idx)

1


In [28]:
mask = ~features.geometry_mask([cShape], data[0, :, :].shape, popRaster.transform)
inData = data[0, :, :][mask]
curPop = np.nansum(inData)
print(curPop)              


820.39526


In [65]:
# Mask the input population data to only include pixels above the density threshold
data = popRaster.read()[0,:,:]
# Generate an urban mask based solely on the density threshold
urbanData = (data > densVal) * 1
urbanData = urbanData.astype("int16")
# Use a sieve to remove small urban clusters that cannot meet the total population threshold
sieve_size = round(totalPopThresh/data.max()) # the minimum number of pixels to reach the totalPopThresh at max density
urban_raster_sieved = features.sieve(urbanData, size=sieve_size)
# Create the final urban population raster to be used in the data calculations
urban_pop_raster = urban_raster_sieved * data
all_shps = []
for cShape, value in features.shapes(urban_raster_sieved, mask=urban_raster_sieved.astype(bool), transform=popRaster.transform):
    all_shps.append(shape(cShape))


In [71]:
from rasterstats import zonal_stats

In [72]:
potential_urban_areas = gpd.GeoDataFrame(geometry=all_shps, crs=popRaster.crs)

urban_pop = zonal_stats(potential_urban_areas, data, affine=popRaster.transform, stats=['sum'], nodata=0)

In [68]:
all_shps[0].__class__

shapely.geometry.polygon.Polygon

In [75]:
potential_urban_areas['pop'] = [up['sum'] for up in urban_pop]
selected_urban_areas = potential_urban_areas[potential_urban_areas['pop'] >= totalPopThresh]
selected_urban_areas.shape

(1946, 2)